In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
%pip install langchain openai chromadb langchainhub tiktoken pypdf langchain-community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.0 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/한국IT서비스학회_Food2Vec을 이용한 개인화된 레시피 추천 서비스.pdf")
documents = loader.load_and_split()
print(len(documents))
print(documents[2])

4
page_content='세부적으로는 , 먼저  음식과  관련된  텍스트  데
이터를  수집한  뒤 해당  데이터로  Food2Vec 을 학
습시킨다. 이때  Food2Vec 임베딩  공간에는  그림  1
에 표기된  ‘음식  상품 ’ 및 ‘요리  레시피 ’ 등이  생
성되며, 학습한  음식  텍스트  데이터  내 ‘단어 ’ 또
한 동일한 임베딩  공간  내에  함께  생성된다. 이에  
그림  3과 같이  유저의  특성  정보를 의미하는  단어  
벡터와, 레시피의  임베딩  벡터  간 유사도를 분석
하여  레시피  별로 개인화된  추천도를  계산한다 . 
 
그림 3 Food2Vec 을 이용한  개인화  추천 예시  
 
3.2. 요리  적합도  모듈  
요리  적합도  모듈은  현재  유저가  보유하고  있
는 음식  상품  및 식재료를  고려하여  각 레시피를  
지금  요리하기 적합한  지의  여부를  판단한다 .  
해당  모듈에서는  먼저 , 각 레시피에서  다양한
이름으로  표기된  동일  식재료  및 음식  상품들을 
동일  대상으로  인식하도록  데이터  전처리  과정을  
적용하였다 . 이후  각 레시피에 기재된  식재료  중 
지금  유저가  가진  식재료  수 비율을  계산하여  각 
레시피  별 요리  적합도  지표로 활용하였 다. 
 
3.3. 레시피  추천  모듈  
레시피  추천  모듈은  Food2Vec 개인화  모듈과  
요리  적합도  모듈의  출력  정보를  종합하여  최종적
으로  추천할  레시피를 결정한다 . 이때 , 유저의  개
인적인  음식  선호도  정보를  더 많이  반영할  것인
지(개인화  모듈 ), 혹은  지금  요리하기  용이한  레시
피를  우선적으로  추천  받을  것인지 (요리  적합도  
모듈 )는 각 유저가  스스로  조정할 수 있도록  구현
하였다 .  
 4. 서비스  구현  및 결과  
 
4.1. 데이터  수집  및 Food2Vec 학습  
Food2Vec 임베딩을  학습하기  위하여 

## RecursiveCharacterTextSplitter
RecursiveCharacterTextSplitter는 문장이나 단락을 기준으로 가능한 한 자연스럽게 텍스트를 나누도록 설계되어 있습니다. 사용자가 구체적인 separators를 지정하면 그에 따라 보다 세밀하게 조정이 가능하지만, 그렇지 않더라도 문장 경계를 고려한 기본적인 동작을 수행하게 됩니다.

이 기본 구분자는 일반적으로 문장 끝에 있는 마침표(.), 개행 문자(\n), 공백( ) 등을 포함하며, 이들 경계를 통해 텍스트를 나누게 됩니다. 이렇게 하면 문장이 갑작스럽게 중단되지 않고 최대한 논리적인 경계를 유지하게 됩니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=15, chunk_overlap=10)
texts = text_splitter.split_text("문장 A. 문장 B. ?? ? 문장 C. 문장 D. 문장 E.")
print(len(texts))
for text in texts:
    print(text)
    print("======")



5
문장 A. 문장 B. ??
문장 B. ?? ? 문장
?? ? 문장 C. 문장
문장 C. 문장 D. 문장
문장 D. 문장 E.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=15, chunk_overlap=10, separators=["\n\n", "\n", "(?<=\. )"," ", ""])
texts = text_splitter.split_text("문장 A. 문장 B. ?? ? 문장 C. 문장 D. 문장 E.")
print(len(texts))
for text in texts:
    print(text)
    print("======")


5
문장 A. 문장 B. ??
문장 B. ?? ? 문장
?? ? 문장 C. 문장
문장 C. 문장 D. 문장
문장 D. 문장 E.


<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
/var/folders/rf/g5t2kw6n1d7db5gnyn05k6f40000gn/T/ipykernel_23179/3139810191.py:3: SyntaxWarning: invalid escape sequence '\.'
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=15, chunk_overlap=10, separators=["\n\n", "\n", "(?<=\. )"," ", ""])


In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))
print(texts[3].page_content)

19
최근  요리와  관련된  AI 연구는 주로  음식  사진_______________________________________  
† 교신저자 (Corresp onding Author)  : CJ올리브네트웍스  
AI-CORE연구소  소장 손종수 (jongsoo.sohn@cj.net )


## vector_store.similarity_search()
- 문서 유사도를 기반으로 한 검색
- 모든 문서의 임베딩 간의 코사인 유사도를 비교

In [58]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(texts, embeddings)
result = vector_store.similarity_search_with_relevance_scores("레시피 추천 서비스에 대해 설명해주세요.")

for r in result:
    print(r)
    print(r[0])
    print("======")
    print(r[1])
    print("======")
    print("======")

(Document(metadata={'page': 0, 'source': 'data/한국IT서비스학회_Food2Vec을 이용한 개인화된 레시피 추천 서비스.pdf'}, page_content='각 유저 별로 개인화된  레시피를 추천할 수 있도록  설계하였다. 본 논문에서  소개하는  서비스는  \n약 1300여 종의 레시피를  추천할  수 있으며 , 향후 실 서비스  적용을  통해 비대면  레시피  추천 서\n비스로  활용될  것으로  기대한다 . \n \n \n1. 서론  \n 코로나 19로 인하여  가정에서의  식사  빈도가  \n증가하고 , 건강  관리  및 실내  취미  활동이  주목받\n으면서  가정에서  직접  요리를 하는  인구가 증가하\n고 있다 . 이와  함께 , 요리  레시피를  공유하는  커뮤니티  및 영상  컨텐츠  분야가  지속적으로  성장하였\n으며 , 또한  간편하게  요리를  할 수 있는  밀키트  \n및 HMR 제품  시장도 크게  성장하였다 . 이에 , 가\n정에서 직접  요리를  하는  고객에게  편의를  제공할  \n수 있는  신규  AI 기술  및 서비스  개발이  다수  진\n행되고  있다 [1,2,3,4,5] .'), 0.8145870782686928)
page_content='각 유저 별로 개인화된  레시피를 추천할 수 있도록  설계하였다. 본 논문에서  소개하는  서비스는  
약 1300여 종의 레시피를  추천할  수 있으며 , 향후 실 서비스  적용을  통해 비대면  레시피  추천 서
비스로  활용될  것으로  기대한다 . 
 
 
1. 서론  
 코로나 19로 인하여  가정에서의  식사  빈도가  
증가하고 , 건강  관리  및 실내  취미  활동이  주목받
으면서  가정에서  직접  요리를 하는  인구가 증가하
고 있다 . 이와  함께 , 요리  레시피를  공유하는  커뮤니티  및 영상  컨텐츠  분야가  지속적으로  성장하였
으며 , 또한  간편하게  요리를  할 수 있는  밀키트  
및 HMR 제품  시장도

## vector_store.as_retriever()
- 검색 결과를 langchain에서 제공하는 다른 시스템과 결합하거나 더 복잡한 시스템에 활용 가능
- 다양한 최적화, 검색 기준 적용 가능
- 검색 개수 및 필터링 가능

In [52]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(texts, embeddings, persist_directory="rag/chroma_db1")
retriever = vector_store.as_retriever()
retriever.invoke("레시피 추천 서비스에 대해 설명해주세요.")


[Document(metadata={'page': 0, 'source': 'data/한국IT서비스학회_Food2Vec을 이용한 개인화된 레시피 추천 서비스.pdf'}, page_content='각 유저 별로 개인화된  레시피를 추천할 수 있도록  설계하였다. 본 논문에서  소개하는  서비스는  \n약 1300여 종의 레시피를  추천할  수 있으며 , 향후 실 서비스  적용을  통해 비대면  레시피  추천 서\n비스로  활용될  것으로  기대한다 . \n \n \n1. 서론  \n 코로나 19로 인하여  가정에서의  식사  빈도가  \n증가하고 , 건강  관리  및 실내  취미  활동이  주목받\n으면서  가정에서  직접  요리를 하는  인구가 증가하\n고 있다 . 이와  함께 , 요리  레시피를  공유하는  커뮤니티  및 영상  컨텐츠  분야가  지속적으로  성장하였\n으며 , 또한  간편하게  요리를  할 수 있는  밀키트  \n및 HMR 제품  시장도 크게  성장하였다 . 이에 , 가\n정에서 직접  요리를  하는  고객에게  편의를  제공할  \n수 있는  신규  AI 기술  및 서비스  개발이  다수  진\n행되고  있다 [1,2,3,4,5] .'),
 Document(metadata={'page': 3, 'source': 'data/한국IT서비스학회_Food2Vec을 이용한 개인화된 레시피 추천 서비스.pdf'}, page_content='4.3. 개인화된 레시피  추천  결과  \n본 논문에서  소개한  레시피  추천  서비스의  실\n행 예시를 표 3에 기재하였다 . 표 3 에서  1, 2행 / \n3, 4행을 비교할 경우 , 동일한  유저임에도  보유  재\n료에  따라  추천되는  레시피가  크게  달라지는  것을  \n확인할  수 있다 . 또한  표 3 의 1, 3행 / 2, 4행을  각\n각 비교할  경우에는  동일한  식재료를  보유하고  있\n는 상황임에도  유저의  특성  변화에  따라  새로운  \n레시피가 추천되거나 , 기존에 추천

## 중복된 내용이 검색되는 이유
- 코드를 여러번 실행하다 보면 출력 결과에 첫 번째, 두 번째 결과의 내용이 중복된다.
- 이는 실행할 때마다 ChromaDB에 새로 embedding을 하고 추가하기 때문이다.
- persist_directory를 지정하지 않으면 Chroma.from_documents()는 데이터를 메모리(RAM)에 저장하게 되는데, 이러면 메모리에 중복되어서 계속 임베딩되고 있는 것이다.
- 이 경우, 프로그램이 종료되면 데이터가 유지되지 않으며, 저장된 내용은 모두 손실되기 때문에 재시작을 해주면 중복해서 나오는 문제가 해결된다.
- 하지만 휘발성 저장소처럼 동작하기 때문에 재사용이 어려워 프로젝트를 진행할 때는 persist_directory를 지정해주는 것이 좋다.